# Basic analysis

NOTE: This requires a python virtualenv already installed with the appropriate dependencies, by e.g.:

`conda create -n acanaba python=3.10 numpy pandas ipykernel jupyter holoviews bokeh datashader seaborn pytables`

Let's load our tracked data and calculate some basic metrics for the pre-reversal and first reversal session.

First we need to import the `pandas` module which allows us to manipulate tabular data. The module provides two basic data structures, DataFrames (table-like) and Series (row-like).

In [ ]:
import pandas as pd
idx = pd.IndexSlice

The `idx` object allows us to do complex indexing on our DataFrames and Series objects.

Now let's load our analysed position data. Initially let's just load up the first data file and see what it looks like.

In [ ]:
track_fns = {'prerev': '../rawdata/sub-lizzy/sub-lizzy_ses-prerev_task-RR10_vidDLC_resnet50_dlc_acan_masterOct23shuffle1_800000.h5',
             'rev01': '../rawdata/sub-lizzy/sub-lizzy_ses-rev01_task-RR10_vidDLC_resnet50_dlc_acan_masterOct23shuffle1_800000.h5'}
_df = pd.read_hdf(track_fns['prerev'])
_df

This is reasonable, but there seems to be an uninformative (and incorrect) 'scorer' index, and the frame number has no name. Let's fix this.

In [ ]:
_df.columns = _df.columns.droplevel(0)
_df.index.name = 'frame_id'
_df

Great, this looks better. Now let's make a function for this, so we can load all our data together.

In [ ]:
def load_track_session(filename: str) -> pd.DataFrame:
    '''Load one session analysed by DeepLabCut into a DataFrame'''
    df = pd.read_hdf(filename)
    df.columns = df.columns.droplevel(0)
    df.index.name = 'frame_id'
    return df

Now load all the sessions into a single DataFrame.

In [ ]:
track_dfs = {key: load_track_session(fn) for key, fn in track_fns.items()}
track_df = pd.concat(track_dfs, names=['session_id'])
track_df

If we want to extract one point, let's say the animal's nose, we can index just those columns.

In [ ]:
track_df.loc[:, idx['nose']]

Similarly, if we want to see just the frames where DeepLabCut classified a point with high confidence, we can filter the results on the likelihood.

In [ ]:
track_df.loc[:, idx['nose']].query('likelihood > 0.95')

## Load the session timings.

We can filter all our positional data by the first and last LED flash in each acquisition run. These values are stored in JSON files associated with each MedPC session.

In [ ]:
from pathlib import Path
import json

In [ ]:
def load_sidecar(filename: str) -> pd.Series:
    with open(filename) as sidecar:
        info = json.load(sidecar)
    return pd.Series(info)

In [ ]:
sidecar_fns = Path('../rawdata').glob('*/*.json')
info_df = pd.DataFrame([load_sidecar(fn) for fn in sidecar_fns]).set_index(['sub', 'ses', 'acq'])
info_df

Now we can label the tracking data with the session and acquisition information.

First let's look at how we can select the correct data for each acquisition session.

In [ ]:
track_df.loc[idx['rev01', 22647:43448], :]

In [ ]:
def get_acq_session(info):
    df = track_df.loc[idx[info.ses, info.firston:info.laston], :]
    df = df.assign(acq=info.acq).set_index('acq', append=True)
    df.index = df.index.reorder_levels(['session_id', 'acq', 'frame_id'])
    return df

In [ ]:
acq_df = pd.concat([get_acq_session(info) for info in info_df.reset_index().itertuples()])
acq_df

In [ ]:
import holoviews as hv
from holoviews import opts
import datashader as ds
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
import panel
panel.extension(comms='vscode')

In [ ]:
nose_df = acq_df.loc[idx['rev01', 'LP', :], idx['nose']].query('likelihood > 0.95')
datashade(hv.Path(nose_df.loc[nose_df['likelihood'] > 0.95])).opts(width=600, height=600)

In [ ]:
paths = {(ses, acq): hv.Path(acq_df.loc[idx[ses, acq, :], idx['nose']].query('likelihood > 0.95'))
         for i, ses, acq in info_df.reset_index().loc[:, ['ses', 'acq']].itertuples()}

In [ ]:
hv.HoloMap(paths, kdims=['ses', 'acq']).layout().cols(2).opts(width=400, height=400)

What about the centre point of the head? We need the mean point between the two ears, filtered for only those points where the likelihoods of both points are above a threshold.

In [ ]:
head_centre_mask = (acq_df.stack().loc[idx[:, :, :, 'likelihood'], idx['leftEar', 'rightEar']] > 0.95).all(axis=1).to_numpy()
head_centre_mask

In [ ]:
head_centre_df = acq_df.stack().loc[idx[:, :, :, ['x', 'y']], ['leftEar', 'rightEar']].mean(axis=1).unstack()
head_centre_df

In [ ]:
paths = {(ses, acq): hv.Path(head_centre_df.loc[idx[ses, acq, head_centre_mask], :])
         for i, ses, acq in info_df.reset_index().loc[:, ['ses', 'acq']].itertuples()}
hv.HoloMap(paths, kdims=['ses', 'acq']).layout().cols(2)

We're plotting trajectories, but what about dwell times? Occupancy maps can give us dwell time.

In [ ]:
import numpy as np
import spatial
bins = np.array((40, 40))
minmax = [head_centre_df.loc[head_centre_mask, ['x', 'y']].min(),
          head_centre_df.loc[head_centre_mask, ['x', 'y']].max()]
path_range = [(a, b) for a, b in zip(*minmax)]
head_centre_df['t'] = head_centre_df.index.get_level_values('frame_id') / 30
_df = head_centre_df.loc[head_centre_mask, ['t', 'x', 'y']]
occ = spatial.occupancy_map(_df.loc[idx['prerev', 'LG', :]].to_numpy(),
                            bins=bins, smooth=1, max_dt=0.2,
                            range=path_range)
hv.Image(occ.hist.T).opts(cmap='viridis', frame_height=400,
                          data_aspect=(bins[1]/bins[0]))